In [1]:
from ipyparallel import Client
client = Client()
view = client.load_balanced_view()


client[:]

<DirectView [0, 1, 2, 3]>

In [14]:
import numpy as np
import pandas as pd

import pims

from PIL import Image, ImageDraw
import cv2

import numba
import timeit

import random

#import trackpy.diag


In [3]:
%%px
import trackpy as tp
tp.quiet()

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

In [4]:
@pims.pipeline
def crop(image, x1, y1, x2, y2):
    npImage = np.array(image)
    
    # Create same size alpha layer with circle
    alpha = Image.new('L', (720, 720), 0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)

    # Convert alpha Image to numpy arrayf
    npAlpha = np.array(alpha)
    npImage = npImage[:,:,1] * npAlpha
    
    ind = np.where(npImage == 0)
    # npImage[150, 150] color of the border to swap with the black
    npImage[ind] = npImage[150, 150]
    return npImage

@pims.pipeline
def crop2(image, x1, y1, x2, y2):    
    npImage = np.array(image)
    # Create same size alpha layer with circle
    #alpha = Image.new('L', (920, 960), 0)
    alpha = Image.new('L', (920, 960), 0)

    draw = ImageDraw.Draw(alpha)
    draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)

    # Convert alpha Image to numpy arrayf
    npAlpha = np.array(alpha)
    npImage = npImage[:, :, 1] * npAlpha
    
    ind = np.where(npImage == 0)
    # npImage[200, 200] color of the border to swap with the black
    npImage[ind] = npImage[200, 200]
    return npImage

In [5]:
frames = crop2(pims.open('/Users/matteoscandola/thesisData/49b-1r-20pPF-25E-4pwt-1.mp4'), 55, 55, 880, 880)
print(frames)
#frames = crop(pims.open('/Users/matteoscandola/thesisData/Movie32.mp4'), 30, 30, 650, 650)
ref = pims.open('/Users/matteoscandola/thesisData/49b-1r-20pPF-25E-4pwt-1.mp4')
oof = crop2(pims.open('/Users/matteoscandola/thesisData/untitled.mp4'), 55, 55, 880, 880)

deprecated pixel format used, make sure you did set range correctly


(PyAVReaderTimed,) processed through proc_func. Original repr:
    <Frames>
    Format: H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10
    Source: /Users/matteoscandola/thesisData/49b-1r-20pPF-25E-4pwt-1.mp4
    Duration: 8079.400 seconds
    Frame rate: 10.000 fps
    Length: 80794 frames
    Frame Shape: (960, 920, 3)
    


deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly


In [6]:
dropSize = 31 #yranges of a drop 335-307 #xranges of a drop 118-90 --> 28 but needs to be odd so 29
minMass = 2000
sep = 16
nDrops = 50

In [10]:
curried_locate = lambda image: tp.locate(image, 31, minmass = 2000, separation = 16, topn = 50)

In [8]:
%%timeit
amr = view.map_async(curried_locate, frames[:10])
amr.wait_interactive()
results = amr.get()

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

<lambda>:   0%|          | 0/10 [00:00<?, ?tasks/s]

2.21 s ± 51.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
amr = view.map_async(curried_locate, frames[:100])
amr.wait_interactive()
results = amr.get()

features_ipy = pd.concat(results, ignore_index=True)
features_ipy

deprecated pixel format used, make sure you did set range correctly
 (repeated 99 more times)
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly


<lambda>:   0%|          | 0/100 [00:00<?, ?tasks/s]

,y,x,mass,size,ecc,signal,raw_mass,ep
0,171.735232,417.068536,6831.555216,10.518919,0.017850,12.622410,70005.0,0.001334
1,180.518298,515.758085,6759.595680,10.346419,0.008272,12.858343,69442.0,0.001373
2,180.531254,573.879268,6605.177594,10.440808,0.035298,13.094276,69837.0,0.001345
3,186.420694,325.681975,7053.686046,10.437935,0.035008,13.330209,71302.0,0.001251
4,219.088718,633.619079,6906.345947,10.383727,0.029452,13.094276,68518.0,0.001443
...,...,...,...,...,...,...,...,...
4995,737.637262,274.282211,9708.786192,10.429192,0.031124,17.868260,110000.0,0.000816
4996,780.353650,425.134796,7711.795365,10.543839,0.029395,13.613913,102858.0,0.001011
4997,778.640848,618.410599,7458.965557,10.440081,0.054054,14.343230,103345.0,0.000995
4998,807.413504,320.699326,8437.465536,10.448895,0.032088,15.680310,105689.0,0.000924
